In [1]:
import cudf
import json

# JSON 파일 경로 설정
file_path = 'merged_data.json'
parquet_file_path = 'merged_data.parquet'

# JSON 파일을 읽어서 리스트로 변환
with open(file_path, 'r', encoding='utf-8') as file:
    json_data = json.load(file)

# jq_schema에 맞춰 데이터를 처리
data = []
for item in json_data:
    processed_item = {
        "filename": item.get("filename"),
        "date": item.get("date"),
        "conference_number": item.get("conference_number"),
        "question_number": item.get("question_number"),
        "meeting_name": item.get("meeting_name"),
        "generation_number": item.get("generation_number"),
        "committee_name": item.get("committee_name"),
        "meeting_number": item.get("meeting_number"),
        "session_number": item.get("session_number"),
        "agenda": item.get("agenda"),
        "law": item.get("law"),
        "qna_type": item.get("qna_type"),
        "context": item.get("context"),
        "context_summary_summary_q": item.get("context_summary", {}).get("summary_q"),
        "context_summary_summary_a": item.get("context_summary", {}).get("summary_a"),
        "questioner_name": item.get("questioner_name"),
        "questioner_affiliation": item.get("questioner_affiliation"),
        "questioner_position": item.get("questioner_position"),
        "question_comment": item.get("question", {}).get("comment"),
        "question_keyword": item.get("question", {}).get("keyword"),
        "answerer_name": item.get("answerer_name"),
        "answerer_affiliation": item.get("answerer_affiliation"),
        "answerer_position": item.get("answerer_position"),
        "answer_comment": item.get("answer", {}).get("comment"),
        "answer_keyword": item.get("answer", {}).get("keyword"),
    }
    data.append(processed_item)

# GPU에서 처리하기 위해 cuDF의 DataFrame으로 변환
df = cudf.DataFrame(data)

# Parquet 파일로 저장
df.to_parquet(parquet_file_path)
